# A full factorial design in Python from Beginning to End

*Adapted into a Jupyter Notebook from the article on Experimental Design Hub.*


### Introduction

Commercial DoE and statistical software is incredibly powerful, but they also come with a hefty price tag. In addition, such software can be pretty daunting because of their many features of which you 80% probably don’t need.

Therefore, this notebook shows how to use Python to perform a **2‑level full factorial design** from start to finish. Feel free to use and adapt the functions to your needs.


### Packages

In [ ]:

# Core packages used throughout the notebook
from pyDOE2 import fullfact
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
from statsmodels.formula.api import ols
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 - needed for 3D plotting

# Display plots inline
%matplotlib inline



**What these packages do (briefly):**

1. `pyDOE2.fullfact` generates full factorial designs.  
2. `numpy` and `pandas` handle arrays and data tables.  
3. `matplotlib` creates visualizations.  
4. `itertools` and `math` assist with combinations and layout math.  
5. `statsmodels` fits linear models (OLS), runs ANOVA, and provides diagnostic tools.  
6. `mpl_toolkits.mplot3d` enables basic 3D plots.


### Experimental plan

In [ ]:

def create_full_factorial_design(factors, randomize=False):
    """Create a 2-level full factorial design (-1/+1 coding).

    Parameters
    ----------
    factors : list[str]
        Names of factors (columns) in the design matrix.
    randomize : bool, optional
        If True, shuffle the order of runs.

    Returns
    -------
    pd.DataFrame
        Design matrix with -1/+1 coded levels and columns named after `factors`.
    """
    # Create a 2-level full factorial design with 0/1 coding
    design = fullfact([2] * len(factors))

    # Convert levels from 0/1 to -1/+1
    design = 2 * design - 1

    # Convert design matrix to a DataFrame
    df = pd.DataFrame(design, columns=factors)

    # Randomize the design if needed
    if randomize:
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    return df



**Example usage:** create a plan for Temperature (T), Pressure (P), Concentration (CoF), and RPM.  
Uncomment the `to_excel` line if you want to export the plan and later record your results in the same file.


In [ ]:

factors = ['T', 'P', 'CoF', 'RPM']
design_df = create_full_factorial_design(factors, randomize=False)
design_df.head()
# design_df.to_excel('full_factorial_design_filtration_rate.xlsx', index=False)



### Visualization

After running your experiments and adding the response (e.g., `Filtration_rate`) as a new column to your Excel/CSV, you can compute **main effects** and **interaction plots** to get a quick overview.

#### Main effects


In [ ]:

def main_effects_plot(excel_file, result_column):
    """Plot main effects for each factor in a -1/+1 coded factorial design.

    Parameters
    ----------
    excel_file : str
        Path to an Excel file containing the design + result.
    result_column : str
        Name of the response column.
    """
    # Load data from excel
    df = pd.read_excel(excel_file)

    # Identify factors by excluding the result column
    factors = [col for col in df.columns if col != result_column]

    # Calculate main effects
    main_effects = {}
    for factor in factors:
        mean_plus = df[df[factor] == 1][result_column].mean()
        mean_minus = df[df[factor] == -1][result_column].mean()
        main_effects[factor] = mean_plus - mean_minus

    # Plot main effects
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.bar(main_effects.keys(), main_effects.values())

    # Annotate the bars with their values
    for factor, value in main_effects.items():
        ax.text(factor, value + 0.01 * abs(value), f"{value:.2f}", ha='center', va='bottom')

    ax.set_ylabel('Main Effect')
    ax.set_title('Main Effects Plot')
    plt.xticks(rotation=45, ha="right")
    plt.show()

# Example call (requires a file with results)
# excel_file = 'full_factorial_design_filtration_rate_results.xlsx'
# result_column = 'Filtration_rate'
# main_effects_plot(excel_file, result_column)


#### Interactions

In [ ]:

def interaction_point_plot(excel_file, result_column):
    """Create point plots for all 2-way interactions.

    Parameters
    ----------
    excel_file : str
        Path to an Excel file containing the design + result.
    result_column : str
        Name of the response column.
    """
    df = pd.read_excel(excel_file)
    factors = [col for col in df.columns if col != result_column]
    interactions = list(itertools.combinations(factors, 2))

    # Calculate number of rows and columns for subplots
    cols = 3
    rows = math.ceil(len(interactions) / cols)
    fig, axs = plt.subplots(rows, cols, figsize=(10, 10 * rows / 3))

    for idx, interaction in enumerate(interactions):
        row = idx // cols
        col = idx % cols

        ax = axs[row, col] if rows > 1 else axs[col]
        for level in [-1, 1]:
            subset = df[df[interaction[0]] == level]
            ax.plot(
                subset[interaction[1]].unique(),
                subset.groupby(interaction[1])[result_column].mean(),
                'o-',
                label=f'{interaction[0]} = {level}'
            )

        ax.set_title(f'{interaction[0]} x {interaction[1]}')
        ax.legend()
        ax.grid(True)

    # Handle any remaining axes (if there's no data to plot in them)
    for idx in range(len(interactions), rows * cols):
        row = idx // cols
        col = idx % cols
        axs[row, col].axis('off')

    plt.tight_layout()
    plt.show()

# Example call (requires a file with results)
# interaction_point_plot(excel_file, result_column)


### Model building / ANOVA

In [ ]:

# Example workflow for model fitting and ANOVA (requires your results file)
# excel_file = 'full_factorial_design_filtration_rate_results.xlsx'

# df = pd.read_excel(excel_file)

# Define a model formula (adjust factors/interactions for your case)
# formula = 'Filtration_rate ~ T + CoF + P + RPM + T:CoF + T:RPM'
# model = ols(formula, data=df).fit()

# Perform ANOVA (Type I SS shown here; consider Type II/III as needed)
# anova_table = sm.stats.anova_lm(model, typ=1)
# anova_table


### Model control / diagnostics

In [ ]:

def diagnostic_plots(model):
    """Create residual diagnostics: residuals vs predicted, residuals vs run, and QQ-plot."""
    # Extract residuals and predicted values from the model
    residuals = model.resid
    predicted = model.fittedvalues

    # Create subplots
    fig, axs = plt.subplots(1, 3, figsize=(10, 5))

    # Residuals vs Predicted
    axs[0].scatter(predicted, residuals, edgecolors='k', facecolors='none')
    axs[0].axhline(y=0, color='k', linestyle='dashed', linewidth=1)
    axs[0].set_title('Residuals vs. Predicted')
    axs[0].set_xlabel('Predicted values')
    axs[0].set_ylabel('Residuals')

    # Residuals vs. Runs (Order of Data Collection)
    axs[1].scatter(range(len(residuals)), residuals, edgecolors='k', facecolors='none')
    axs[1].axhline(y=0, color='k', linestyle='dashed', linewidth=1)
    axs[1].set_title('Residuals vs. Run')
    axs[1].set_xlabel('Run')
    axs[1].set_ylabel('Residuals')

    # Q-Q plot
    sm.qqplot(residuals, line='45', fit=True, ax=axs[2])
    axs[2].set_title('Q-Q Plot')

    plt.tight_layout()
    plt.show()

# Example:
# print(model.summary())
# diagnostic_plots(model)


### 3D plots

In [ ]:

def plot_3D_surface(model, data, x_name, y_name, z_name, held_factor, held_value, title):
    """Plot a 3D surface for two factors while holding a third constant."""
    x_range = np.linspace(-1, 1, 100)
    y_range = np.linspace(-1, 1, 100)
    x_grid, y_grid = np.meshgrid(x_range, y_range)

    df_pred = pd.DataFrame({
        x_name: x_grid.ravel(),
        y_name: y_grid.ravel(),
        held_factor: [held_value] * x_grid.size
    })
    df_pred = sm.add_constant(df_pred)  # Add a constant to match the model's expectations
    Z = model.predict(df_pred).values.reshape(x_grid.shape)

    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Surface
    ax.plot_surface(x_grid, y_grid, Z, cmap='viridis', alpha=0.7)

    # Overlay observed points at the selected held value
    mask = data[held_factor] == held_value
    ax.scatter(data.loc[mask, x_name], data.loc[mask, y_name], data.loc[mask, z_name], edgecolor='k')

    ax.set_xlabel(x_name)
    ax.set_ylabel(y_name)
    ax.set_zlabel(z_name)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

# Example (requires fitted `model` and your dataframe `df` with columns `[T, P, CoF, RPM, Filtration_rate]`):
# plot_3D_surface(model, df, 'T', 'CoF', 'Filtration_rate', held_factor='RPM', held_value=1, title='T vs CoF @ RPM=+1')


### Contour plots

In [ ]:

# Function to create contour plot
def plot_contour(model, x_name, y_name, held_factor, held_value, title):
    x_range = np.linspace(-1, 1, 100)
    y_range = np.linspace(-1, 1, 100)
    x_grid, y_grid = np.meshgrid(x_range, y_range)

    predictions = model.predict(pd.DataFrame({
        x_name: x_grid.ravel(),
        y_name: y_grid.ravel(),
        held_factor: held_value,
    }))
    Z = predictions.values.reshape(x_grid.shape)

    plt.figure(figsize=(7, 5))
    contour = plt.contourf(x_grid, y_grid, Z, 20, cmap='viridis')
    plt.colorbar(contour)
    plt.title(title)
    plt.xlabel(x_name)
    plt.ylabel(y_name)
    plt.tight_layout()
    plt.show()

# Example (requires fitted `model`):
# plot_contour(model, 'T', 'CoF', held_factor='RPM', held_value=1, title='Contour of T vs CoF @ RPM=+1')



---

### Notes
- The plotting functions assume **-1/+1 coding** for factor levels.
- Replace file names, factor names, and the **model formula** with those relevant to your experiment.
- Consider using Type II (`typ=2`) or Type III (`typ=3`) ANOVA if you have an unbalanced design or want different sums of squares.

*Source: Adapted from “A full factorial design in Python from Beginning to End” on Experimental Design Hub.*
